## Calculations on variables

In [1]:
%load_ext autoreload
%autoreload

import iris
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
dir = '../outputs/stash_out/'

files = {'fractional_cover' : 'vegcover1929oct.nc',
         'alpha'            : 'alpha1929oct.nc',
         'relative_humidity': 'relative_humidity1929oct.nc',
         'lightning'        : 'lightning1929oct.nc'}

tree =[101, 102]

### Loading in the data

In [4]:
input_data = {}
for key, file in files.items():
    data = iris.load_cube(dir + file)
    input_data[key] = data

In [62]:
input_data['fractional_cover'].coord('pseudo_level').points==201 #| 202

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [3]:
# Loading in data from year 2000, ap5
dir = '../data/UKESM/historic_1/'
year = '2000'
files = []
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']


for month in months:
    files.append('bc179a.p5' + year + month +'.pp')
 

print(files)

['bc179a.p52000jan.pp', 'bc179a.p52000feb.pp', 'bc179a.p52000mar.pp', 'bc179a.p52000apr.pp', 'bc179a.p52000may.pp', 'bc179a.p52000jun.pp', 'bc179a.p52000jul.pp', 'bc179a.p52000aug.pp', 'bc179a.p52000sep.pp', 'bc179a.p52000oct.pp', 'bc179a.p52000nov.pp', 'bc179a.p52000dec.pp']


Here is some of Doug's code to help extract the correct layers of fractional cover

In [16]:
files = 'bc179a.p51929oct.pp'
dir = '../data/'

stash_constraint = iris.AttributeConstraint(STASH = "m01s03i317")
cube = iris.load_cube(dir + files, stash_constraint)

#index = np.where(cube.coord('pseudo_level').points)
index = [cube.coord('pseudo_level').points == x for x in tree]
index = np.any(index, axis = 0) # This combines all the boolean arrays together. True + False = True
#print(index)
#print(cube.coord('pseudo_level')[index])
print(cube[index])

m01s03i317 / (unknown)              (pseudo_level: 2; latitude: 144; longitude: 192)
     Dimension coordinates:
          pseudo_level                           x            -               -
          latitude                               -            x               -
          longitude                              -            -               x
     Scalar coordinates:
          forecast_period: 689400.0 hours, bound=(689040.0, 689760.0) hours
          forecast_reference_time: 1850-01-01 00:00:00
          time: 1929-10-16 00:00:00, bound=(1929-10-01 00:00:00, 1929-11-01 00:00:00)
     Attributes:
          STASH: m01s03i317
          source: Data from Met Office Unified Model
          um_version: 10.9
     Cell methods:
          mean: time (1 hour)


In [25]:
files = 'bc179a.p51929oct.pp'
dir = '../data/'

stash_constraint = iris.AttributeConstraint(STASH = "m01s08i223")
cube = iris.load_cube(dir + files, stash_constraint)

#index = np.where(cube.coord('pseudo_level').points)
index = [cube.coord('depth').points == 0.05]
index = np.any(index, axis = 0) # This makes the cube happy
print(cube[index])

[ True False False False]
moisture_content_of_soil_layer / (kg m-2) (depth: 1; latitude: 144; longitude: 192)
     Dimension coordinates:
          depth                                 x            -               -
          latitude                              -            x               -
          longitude                             -            -               x
     Scalar coordinates:
          forecast_period: 689400.0 hours, bound=(689040.0, 689760.0) hours
          forecast_reference_time: 1850-01-01 00:00:00
          time: 1929-10-16 00:00:00, bound=(1929-10-01 00:00:00, 1929-11-01 00:00:00)
     Attributes:
          STASH: m01s08i223
          source: Data from Met Office Unified Model
          um_version: 10.9
     Cell methods:
          mean: time (1 hour)


### Trying to get alphaMax

In [4]:
stash_constraint = iris.AttributeConstraint(STASH = 'm01s08i223')

# Load all cubes
aList =[]
cube_list = iris.cube.CubeList()
for f in files: 
    dat = iris.load_cube(dir + f, stash_constraint)
    aList.append(dat)

# Merge all cubes together
cube_list = iris.cube.CubeList(aList)
cube_alpha_new = cube_list.merge_cube() 

# Extract just the top soil
index_soil = [cube_alpha_new.coord('depth').points == 0.05]
index_soil = np.any(index_soil, axis = 0) # Still keep this in - it makes the cube happy
cube_soil = cube_alpha_new[:, index_soil]

In [5]:
print(cube_soil[6:,:,:,:])

moisture_content_of_soil_layer / (kg m-2) (time: 6; depth: 1; latitude: 144; longitude: 192)
     Dimension coordinates:
          time                                 x         -            -               -
          depth                                -         x            -               -
          latitude                             -         -            x               -
          longitude                            -         -            -               x
     Auxiliary coordinates:
          forecast_period                      x         -            -               -
     Scalar coordinates:
          forecast_reference_time: 1850-01-01 00:00:00
     Attributes:
          STASH: m01s08i223
          source: Data from Met Office Unified Model
          um_version: 10.9
     Cell methods:
          mean: time (1 hour)


Rich's code

In [94]:
# Here, we're taking the 3rd time element all the way to the last time point and put it in a new cube
# Note: we're only taking the first 3 months, because I've currently only uploaded a year's worth of data
cube2 = cube_soil[3:,:,:,:]
cube3 = cube_soil[3:,:,:,:]
cube4 = cube_soil[3:,:,:,:]
cube5 = cube_soil[3:,:,:,:]

nmonths = len(cube2.coord("time").points)
print(*range(nmonths))

# This loop is saying: for the first 3 months of the original cube (cube_soil), take those values, collapse them by the 
# mean, and then take the data and put it into cube2 as its first timepoint (which is 3 months ahead)

for m in range( nmonths):
    cube2.data[m,:,:,:] = cube_soil[m:m+3,:,:,:].collapsed(["time"], iris.analysis.MEAN).data
    cube3.data[m,:,:,:] = cube_soil[m:m+3,:,:,:].collapsed(["time"], iris.analysis.MAX).data
    cube4.data[m,:,:,:] = (cube3.data[m,:,:,:] / cube2.data[m,:,:,:]) - 1
    cube5.data[m,:,:,:] = (cube3[m,:,:,:].data / cube2[m,:,:,:].data) - 1

0 1 2 3 4 5 6 7 8
0
0.13396860205608865
0.13396860205608865
1
0.17919338060461956
0.17919338060461956
2
0.22689228887143342
0.22689228887143342
3
0.23411234980044157
0.23411234980044157
4
0.23328306778617527
0.23328306778617527
5
0.21472162993057914
0.21472162993057914
6
0.19499083809230638
0.19499083809230638
7
0.24470976124639096
0.24470976124639096
8
0.2508361318837041
0.2508361318837041


Print out to see if the values are the same

In [73]:
for m in range(0,6):
    print(m)
    print('cube_soil: ' + str(cube_soil[m:m+3,:,:,:].collapsed(['time', 'depth', 'latitude', 'longitude'], iris.analysis.MEAN).data))
    print('cube2: ' + str(cube2[m,:,:,:].collapsed(['time', 'depth', 'latitude', 'longitude'], iris.analysis.MEAN).data))
    print('cube3: ' + str(cube3[m,:,:,:].collapsed(['time', 'depth', 'latitude', 'longitude'], iris.analysis.MEAN).data))
    print('cube4: ' + str(cube4[m,:,:,:].collapsed(['time', 'depth', 'latitude', 'longitude'], iris.analysis.MEAN).data))

0
cube_soil: 18.597834
cube2: 18.59783403961416
cube3: 19.65778791674471
cube4: 1.1339685854704484
1
cube_soil: 18.266462
cube2: 18.26646346460011
cube3: 19.715636120996443
cube4: 1.1791933806046195
2
cube_soil: 17.594904
cube2: 17.594904535025286
cube3: 19.598848098894923
cube4: 1.2268923552139945
3
cube_soil: 16.533796
cube2: 16.53379612286945
cube3: 18.835416569114066
cube4: 1.2341122834578804
4
cube_soil: 15.3931465
cube2: 15.393147710245364
cube3: 17.632717386214647
cube4: 1.233283001443614
5
cube_soil: 14.438873
cube2: 14.438872565087095
cube3: 16.242471378067055
cube4: 1.2147215470023778


In [86]:
x = np.array([[5,5],[5,5]])
y = np.array([[1,2], [5,3]])
print(x/y)              

[[5.         2.5       ]
 [1.         1.66666667]]
